In [1]:
from microagg1d.wilber import Wilber
import numpy as np
from numba import njit
from numba.experimental import jitclass

In [2]:
from microagg1d.wilber import *
from fast1dkmeans.regularized_kmeans import _smawk_iter

In [3]:
@njit()
def reduce_iter(rows, cols, calculator, next_cols):
    # https://courses.engr.illinois.edu/cs473/sp2016/notes/06-sparsedynprog.pdf

    m = len(rows)
    next_cols[0]=cols[0]
    stack_size = 0
    for k in cols:
        if k != next_cols[stack_size]:
            while stack_size >= 0:
                if rows[stack_size] < next_cols[stack_size]:
                    print("reduce1")
                if rows[stack_size] < k:
                    print("reduce2", rows[stack_size] - k, rows[stack_size], k, rows, cols)
                if calculator.calc(rows[stack_size], next_cols[stack_size]) > calculator.calc(rows[stack_size], k):
                    stack_size-=1
                else:
                    break
        if stack_size < m-1:
            stack_size+=1
            next_cols[stack_size]=k
    return stack_size+1


@njit()
def interpolate(rows, cols, calculator, result):
    curr=0
    for i in range(2, len(rows), 2):
        start_col = result[rows[i-1]]
        if i+1 < len(rows):
            stop_col = result[rows[i+1]]
        else:
            stop_col = cols[len(cols)-1]
        while cols[curr] < start_col:
            curr+=1
        best = curr
        best_val = calculator.calc(rows[i], cols[best])
        if rows[i] < cols[best]:
            print("interpolate1")
        while cols[curr] < stop_col:
            tmp = calculator.calc(rows[i], cols[curr+1])
            if rows[i] < cols[curr+1]:
                print("interpolate2")
            if best_val > tmp:
                best = curr+1
                best_val = tmp

            curr+=1
        result[rows[i]]=cols[best]

@njit
def calc_max_col_space(n_rows, n_cols):
    max_cols = n_cols
    depth=0
    col_starts = [0, n_cols]
    while True:
        row_step_size = 2**depth
        n_cols_this_depth = min(n_rows//row_step_size, n_cols)
        max_cols+=n_cols_this_depth
        col_starts.append(max_cols)
        depth+=1
        if n_cols_this_depth == 0:
            break
    return np.array(col_starts), depth

@njit()
def _smawk_iter(rows_in, cols_in, calculator, result):
    col_starts, max_depth = calc_max_col_space(len(rows_in), len(cols_in))
    col_ends = col_starts[1:].copy()
    #col_buffer= np.empty(col_starts[-1], dtype=cols_in.dtype)
    col_buffer= - np.ones(col_starts[-1], dtype=cols_in.dtype)
    col_buffer[:len(cols_in)]=cols_in
    #print(col_starts)
    #print(col_buffer)
    #print(max_depth)

    depth = 0
    for depth in range(0, max_depth):
        step_size = 2**depth
        rows = rows_in[step_size-1::step_size]
        cols = col_buffer[col_starts[depth]:col_ends[depth]]
        #print(depth, rows, cols)

        if len(rows)==0:
            break
        if len(cols)==1:
            for r in rows:
                result[r]=cols[0]
            break
        S = col_buffer[col_starts[depth+1]:col_ends[depth+1]]
        #print(S)
        col_ends[depth+1] = col_starts[depth+1]+reduce_iter(rows, cols, calculator, S)

        _cols = col_buffer[col_starts[depth+1]:col_ends[depth+1]]
        #print(_cols)
        #print()
        result[rows[0]]=_cols[0]
    #print(col_buffer)
    #print("fill")
    for depth in range(max_depth, -1, -1):
        step_size = 2**depth
        rows = rows_in[step_size-1::step_size]
        #print(rows)
        if len(rows)==0:
            continue
        _cols = col_buffer[col_starts[depth+1]:col_ends[depth+1]]
        #print(cols, "\n")
        if len(rows)>2:
            interpolate(rows, _cols, calculator, result)
    #print(result)

In [4]:
@njit()
def __Wilber(n, wil_calculator):
    """Solves Univariate Microaggregation problem in O(n)
    this is an implementation of the proposed algorithm
    from "The concave least weight subsequence problem revisited" by Robert Wilber 1987
    """
    print("Here")
    F = -np.ones(n, dtype=np.int32) #### TODO: CHANGE to empty
    F_vals = wil_calculator.F_vals
    H = -np.ones(n, dtype=np.int32) #### TODO: CHANGE to empty
    H_vals = -np.ones(n+1, dtype=np.float64)
    F_vals[0]=0
    c = 0 # columns [0,c] have correct F_vals
    r = 0 # rows [r,c] may contain column minima


    while c < n:
        p = min(2*c-r+1, n)
        #print("p", p)
        #print("F_input", r, c+1, c, p)
        _smawk_iter(np.arange(c, p), np.arange(r, c+1), wil_calculator, F)
        #print("F", F)

        for j in range(c, p):
            assert j >= F[j]
            F_vals[j+1] = wil_calculator.calc(j, F[j])
        #print("F_val", F_vals)
        #print("H", c+1, p, c+1,p)
        
        # calc values in triangular matrix assuming F_vals contain the coorect min vals
        _smawk_iter(np.arange(c+1, p), np.arange(c+1, p), wil_calculator, H)
        for j in range(c+1, p):
            assert j >= H[j]
            H_vals[j+1] = wil_calculator.calc(j, H[j])
        #print("H_val", H_vals)
        #print(H)
        #print(H_vals)
        j0=p+1
        for j in range(c+2, p+1):
            #print("<< j",j, H_vals[j], F_vals[j])
            if H_vals[j] < F_vals[j]:
                F[j-1] = H[j-1]
                j0 = j
                break
        if j0==p+1: # we were right all along
            # F_vals up to p (inclusive) are correct
            r = F[p-1]
            c = p
        else: # our guessing strategy failed
            F_vals[j0] = H_vals[j0]
            r = c+1
            c = j0
        #print(F)
        #print(F_vals)
        #print()

    return F


@jitclass([('cumsum', float64[:]), ('cumsum2', float64[:]), ('k', int64), ("F_vals", float64[:]), ("SMALL_VAL", float64), ("LARGE_VAL", float64), ("vals", int64[:])])
class MicroaggWilberCalculator:
    """An educational variant of the microagg calculator which keeps track of all the states visited in matrix G"""
    def __init__(self, cumsum, cumsum2, k, F_vals):
        self.cumsum = cumsum
        self.cumsum2 = cumsum2
        self.k = k
        self.F_vals = F_vals
        n = len(cumsum) - 1
        self.SMALL_VAL = calc_objective_upper_inclusive(cumsum, cumsum2, 0, n-1)
        self.LARGE_VAL = self.SMALL_VAL * (1 + n)
        self.vals = np.zeros(4, dtype=np.int64)

    def calc(self, j, i): # i <-> j interchanged is not a bug!
        if j < i:
            self.vals[0]+=1
            return np.inf

        if not (j+1 - i >= self.k):
            self.vals[1]+=1
            return self.SMALL_VAL * (len(self.cumsum)+ i)
        if not (j+1 - i <= 2 * self.k - 1):
            self.vals[2]+=1
            return self.SMALL_VAL * (len(self.cumsum) - i)
        self.vals[3]+=1
        #print(i, j, calc_objective_upper_inclusive(self.cumsum, self.cumsum2, i, j) + self.F_vals[i] , self.F_vals[i])
        return calc_objective_upper_inclusive(self.cumsum, self.cumsum2, i, j) + self.F_vals[i]

@njit([(float64[:], int64)])
def _Wilber(v, k):
    n = len(v)
    cumsum = calc_cumsum(v)
    cumsum2 = calc_cumsum2(v)
    wil_calculator = MicroaggWilberCalculator(cumsum, cumsum2, k, -np.ones(n+1, dtype=np.float64))

    tmp= relabel_clusters_plus_one(__Wilber(n, wil_calculator))
    
    print(wil_calculator.vals)
    return tmp

def Wilber(arr, k : int):
    """Solves the REGULARIZED 1d kmeans problem in O(n)
    this is an implementation of the proposed algorithm
    from "The concave least weight subsequence problem revisited" by Robert Wilber 1987
    """

    assert k > 0
    assert k <= len(arr)
    res = trivial_cases(len(arr), k)
    if not res is None:
        return res
    return _Wilber(arr, k)

In [5]:

arr = np.arange(1000001, dtype=np.float64)
result = Wilber(arr, 2)
print(result)

Here
[      0 1214903 1906802 5068325]
[     0      0      1 ... 430862 430863 430863]


In [6]:
arr = np.arange(100001, dtype=np.float64)
result = Wilber(arr, 2)
print(result)

Here
[     0 133332 183331 516658]
[    0     0     1 ... 49999 49999 49999]


In [7]:
from microagg1d.main import _simple_dynamic_program

In [8]:
@njit([(float64[:], float64[:], int64, int64, bool_)], cache=USE_CACHE)
def calc_objective_upper_inclusive(cumsum, cumsum2, i, j, val=False):
    """Compute the cluster cost of clustering points including both i and j"""
    if j <= i:
        return 0

    #print("\t", cumsum[j + 1]-cumsum[i])
    mu = (cumsum[j + 1]-cumsum[i])/(j + 1 - i)
    result = cumsum2[j + 1] - cumsum2[i]
    #print("\t", result)
    result -= (2 * mu) * (cumsum[j + 1] - cumsum[i])
    result += (j - i + 1) * (mu * mu)

    return max(result, 0.0)

In [9]:
@njit([(float64[:], float64[:], float64[:], float64[:], int64, int64)], cache=USE_CACHE)
def calc_objective_upper_inclusive_2(i_cumsum, i_cumsum2, j_cumsum, j_cumsum2,  i, j):
    """Compute the cluster cost of clustering points including both i and j"""
    cell_size = len(i_cumsum)-1
    val1 = j_cumsum[j + 1] + i_cumsum[cell_size] - i_cumsum[i]
    #print("\t", val1)
    mu = (val1)/(j + 1 + cell_size - i)
    result = j_cumsum2[j + 1] + i_cumsum2[cell_size] - i_cumsum2[i]
    #print("\t", result)
    result -= (2 * mu) * val1
    result += (j - i + 1+ cell_size) * (mu * mu)
    #print("\t", result)
    return max(result, 0.0)

In [39]:
np.cumsum(np.arange(1,5))

array([ 1,  3,  6, 10])

In [41]:
calc_cumsum_cell(np.arange(1,5,dtype=np.float64), 2)

array([[0., 1., 3.],
       [0., 3., 7.]])

In [42]:
np.array([[0,1,3], [0, 3, 7]])

array([[0, 1, 3],
       [0, 3, 7]])

In [16]:
@jitclass([('cumsum', float64[:]), ('cumsum2', float64[:])])
class CumsumCalculator:
    def __init__(self, v):
        self.cumsum = calc_cumsum(v)
        self.cumsum2 = calc_cumsum2(v)

    def calc(self, i, j):
        if i>476357-40:
            print(j-i, i,j,calc_objective_upper_inclusive(self.cumsum, self.cumsum2, i, j,False))
        return calc_objective_upper_inclusive(self.cumsum, self.cumsum2, i, j,True)


    
@njit([(float64[:],int64)])
def calc_cumsum_cell(v, cell_size):
    quotient, remainder = divmod(len(v), cell_size)
    num_cells = quotient
    if remainder!=0:
        num_cells+=1
    out = np.zeros((num_cells, cell_size+1), dtype=np.float64)

    for i in range(quotient):
        curr_out = out[i,:]
        curr_out[0] = 0.0
        offset = i*cell_size
        for j in range(cell_size):
            x = v[offset+j]
            curr_out[j+1] = curr_out[j] + x
    if remainder != 0:
        i=quotient
        curr_out = out[i,:]
        curr_out[0] = 0.0
        offset = i*cell_size
        for j in range(remainder):
            x = v[offset+j]
            curr_out[j+1] = curr_out[j] + x
    return out
    
@jitclass([('cumsum', float64[:,:]), ('cumsum2', float64[:,:]), ('cell_size', int64)])
class StableCumsumCalculator:
    def __init__(self, v, cell_size):
        self.cumsum = calc_cumsum_cell(v, cell_size)
        self.cumsum2 = calc_cumsum_cell(np.square(v), cell_size)
        self.cell_size = cell_size

    def calc(self, i, j):
        if j==i:
            return 0
        assert j>i
        assert j - i < 2 * self.cell_size

    
        cell_i, remainder_i = divmod(i, self.cell_size)
        cell_j, remainder_j = divmod(j, self.cell_size)
        if cell_i  == cell_j: # both are in one cell
            return calc_objective_upper_inclusive(self.cumsum[cell_i,:], self.cumsum2[cell_i,:], remainder_i, remainder_j, False)
        else:
            return calc_objective_upper_inclusive_2(self.cumsum[cell_i,:], self.cumsum2[cell_i,:], self.cumsum[cell_j,:], self.cumsum2[cell_j,:], remainder_i, remainder_j)
    
@njit
def compute_cluster_cost_sorted(v, clusters_sorted):
    calculator = CumsumCalculator(v)
    s = 0.0
    i = 0
    j = 1
    while j < len(v):
        while clusters_sorted[j]==clusters_sorted[i] and  j < len(v):
            j+=1
        s+=calculator.calc(i, j-1)
        i=j
        j=i+1
    return s

In [38]:
np.repeat(np.arange(5), 2)

array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

In [17]:
@njit
def test(a, b):
    return divmod(a,b)

In [18]:
with np.printoptions(linewidth=200, precision=3, suppress=True):
    print(calc_cumsum_cell(np.arange(13, dtype=np.float64), 5))

[[ 0.  0.  1.  3.  6. 10.]
 [ 0.  5. 11. 18. 26. 35.]
 [ 0. 10. 21. 33.  0.  0.]]


In [19]:
c =  CumsumCalculator(np.arange(13, dtype=np.float64))
for j in range(10):
    print(c.calc(0,j))
print()
for j in range(6, 10):
    print(c.calc(5,j))

0.0
0.5
2.0
5.0
10.0
17.5
28.0
42.0
60.0
82.5

0.5
2.0
5.0
10.0


In [20]:
s = StableCumsumCalculator(np.arange(13, dtype=np.float64), 5)



In [21]:
for j in range(6, 10):
    print(s.calc(5,j))

0.5
2.0
5.0
10.0


In [22]:
for j in range(10):
    print(s.calc(0,j))

0.0
0.5
2.0
5.0
10.0
17.5
28.0
42.0
60.0
82.5


In [23]:
s.cumsum

array([[ 0.,  0.,  1.,  3.,  6., 10.],
       [ 0.,  5., 11., 18., 26., 35.],
       [ 0., 10., 21., 33.,  0.,  0.]])

In [24]:
s.cumsum2

array([[  0.,   0.,   1.,   5.,  14.,  30.],
       [  0.,  25.,  61., 110., 174., 255.],
       [  0., 100., 221., 365.,   0.,   0.]])

In [25]:
@njit()
def calc_num_clusters(result):
    """Compute the number of clusters encoded in results
    Can be used on e.g. _optimal_univariate_microaggregation
    """
    num_clusters = 0
    curr_pos = len(result)-1
    while result[curr_pos]>=0:
        curr_pos = result[curr_pos]
        num_clusters+=1
    return num_clusters+1


@njit()
def relabel_clusters(result):
    num_clusters = calc_num_clusters(result)-1

    out = np.empty_like(result)
    curr_pos = len(result)-1
    while result[curr_pos]>=0:
        out[result[curr_pos]:curr_pos+1] = num_clusters
        curr_pos = result[curr_pos]
        num_clusters-=1
    out[0:curr_pos+1] = num_clusters
    return out

In [26]:
@njit()
def _simple_dynamic_program(x, k):
    n = len(x)
    assert k > 0
    if n//2 < k: # there can only be one cluster
        return np.zeros(n, dtype=np.int64)
    if k==1: # each node has its own cluster
        return np.arange(n, dtype=np.int64)
    calculator = StableCumsumCalculator(x, 2*k)


    back_tracks =  np.zeros(n, dtype=np.int64)
    min_vals = np.zeros(n)
    for i in range(0, k-1):
        min_vals[i] = np.inf
        back_tracks[i]=-1
    for i in range(k-1, 2*k-1):
        min_vals[i] = calculator.calc(0,i)
        back_tracks[i]=-1

    for i in range(2*k-1, n):
        #print("i", i)
        min_index = i-2*k+1
        #print("min", min_index)
        prev_min_val = min_vals[min_index] + calculator.calc(min_index+1, i)
        for j in range(i-2*k + 2, i-k+1):
            #print(j, min_vals[j], prev_min_val)
            new_val = min_vals[j] + calculator.calc(j+1, i)
            if  new_val < prev_min_val:
                min_index = j
                prev_min_val = new_val
        #print("result", min_index, prev_min_val)

        back_tracks[i] = min_index
        min_vals[i] = prev_min_val
        #print(back_tracks)
    return relabel_clusters(back_tracks)

In [44]:
#arr = np.arange(476357+3, dtype=np.float64)
arr = np.arange(500_000, dtype=np.float64)
result = _simple_dynamic_program(arr, 2)
print(result[:10], result[-10:])

[0 0 1 1 2 2 3 3 4 4] [249995 249995 249996 249996 249997 249997 249998 249998 249999 249999]


In [31]:
#@njit([(float64[:],)])
def cumsum2(v):
    out = np.empty(len(v)+1, dtype=np.float64)
    out[0]=0.0
    out[1]=v[0]*v[0]
    s = 1
    for i in range(1, len(v)):
        x = v[i]
        s2 = s*s
        out[i+1] = out[i]/s2 + x*x/s2
        s+= x
    return out

In [32]:
tmp = np.square(np.arange(476357+3, dtype=np.float64))
for i in tmp[-20:]:
    print(i)

226899795600.0
226900748281.0
226901700964.0
226902653649.0
226903606336.0
226904559025.0
226905511716.0
226906464409.0
226907417104.0
226908369801.0
226909322500.0
226910275201.0
226911227904.0
226912180609.0
226913133316.0
226914086025.0
226915038736.0
226915991449.0
226916944164.0
226917896881.0


In [33]:
tmp = cumsum2(np.arange(476357+3, dtype=np.float64))
for i,j in zip(np.diff(tmp[-20:]), tmp[-20:]):
    print(i, j)

-7.401848599136923e-17 1.7629001313691914e-11
-7.401801982339527e-17 1.7628927295205922e-11
-7.401755365865249e-17 1.76288532771861e-11
-7.401708749714087e-17 1.762877925963244e-11
-7.401662134532278e-17 1.7628705242544943e-11
-7.401615518381117e-17 1.7628631225923598e-11
-7.40156890416866e-17 1.7628557209768414e-11
-7.401522289309968e-17 1.7628483194079372e-11
-7.401475675097512e-17 1.762840917885648e-11
-7.401429061854407e-17 1.7628335164099728e-11
-7.401382447965068e-17 1.762826114980911e-11
-7.401335835368198e-17 1.762818713598463e-11
-7.401289222771328e-17 1.7628113122626276e-11
-7.401242610497576e-17 1.762803910973405e-11
-7.401195998870059e-17 1.7627965097307944e-11
-7.401149387242542e-17 1.7627891085347955e-11
-7.401102776261259e-17 1.7627817073854082e-11
-7.401056165926211e-17 1.762774306282632e-11
-7.401009555591164e-17 1.762766905226466e-11


In [34]:
for i, x in enumerate(tmp):
    if x  > 2**53:
        print(i, x)
        break

In [35]:
3.602726299525706e+16 < 2**53

False

In [36]:
c = CumsumCalculator(np.arange(476357+3, dtype=np.float64))
for i in np.diff(c.cumsum2[-20:]):
    print(i)

226900748280.0
226901700964.0
226902653648.0
226903606336.0
226904559024.0
226905511716.0
226906464412.0
226907417104.0
226908369800.0
226909322504.0
226910275200.0
226911227904.0
226912180608.0
226913133312.0
226914086024.0
226915038736.0
226915991448.0
226916944160.0
226917896880.0


In [37]:
c.cumsum2[-20:]

array([3.60272630e+16, 3.60274899e+16, 3.60277168e+16, 3.60279437e+16,
       3.60281706e+16, 3.60283975e+16, 3.60286244e+16, 3.60288513e+16,
       3.60290782e+16, 3.60293051e+16, 3.60295320e+16, 3.60297590e+16,
       3.60299859e+16, 3.60302128e+16, 3.60304397e+16, 3.60306666e+16,
       3.60308935e+16, 3.60311204e+16, 3.60313474e+16, 3.60315743e+16])

In [ ]:
2**53

In [ ]:
2 476346 476348 5.0
	 1429041.0
	 680719393232.0
	mu 476347.0
	 5.0
	2*mu 1361438786454.0
	 680719393227.0

In [ ]:
680719393232-1361438786454 + 3*476347*476347

In [ ]:
680719393232


In [ ]:
val = np.cumsum(np.square(np.arange(1000001, dtype=np.float64)))

In [ ]:
2 476348 476350 5.0
	 1429047.0
	 680725109408.0
mu 476349.0
-680725109398.0
2*mu 1361450218806.0
680725109403.0
5.0

In [ ]:
3*476347*476347

In [ ]:
val[-1]==val[-2]

In [ ]:
for i in range(1, len(result), 2):
    if result[i]!=i//2:
        print(i)
        break

In [ ]:
result[476357-10: 476357+10]

In [ ]:
v = np.arange(1_000_001, dtype=np.float64)
cumsum = calc_cumsum(v)
cumsum2 = calc_cumsum2(v)
d=5
for i in result[476357-d: 476357+d]:
    for j in result[476357-d: 476357+d]:
        res = calc_objective_upper_inclusive(cumsum, cumsum2, i, j) +0.5*238176
        print(i, j, res)